Kerri's code with cross-validation. 

In [ ]:
pip install tf-nightly

     |████████████████████████████████| 345.3MB 42kB/s 
     |████████████████████████████████| 8.9MB 55.6MB/s 
     |████████████████████████████████| 460kB 52.5MB/s 


In [ ]:
pip install tensorflow==2.0.0

     |████████████████████████████████| 86.3MB 36kB/s 
     |████████████████████████████████| 3.8MB 48.5MB/s 
     |████████████████████████████████| 450kB 55.1MB/s 
     |████████████████████████████████| 51kB 8.1MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=a1b4a2604d1f21f943522e6a42ef31dcf121328ffaf1a8da04a1ea5ec4fe7095
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tf-nightly 2.4.0.dev20200819 has requirement gast==0.3.3, but you'll have gast 0.2.2 which is incompatible.
ERROR: tensorflow-probability 0.11.0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3
  Found existing installation: tensorboard 2.3.0
    Uninstalling tensorboard-2.3.0:
      Successfully uninstalled tensorboard-2.3.0
  Found existing install

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals
import os
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import KFold

import tensorflow as tf
keras = tf.keras

AUTOTUNE = tf.data.experimental.AUTOTUNE

import IPython.display as display
from PIL import Image

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!unzip -uq "/content/drive/My Drive/riskmap_urop/none" 
!unzip -uq "/content/drive/My Drive/riskmap_urop/mild" 
!unzip -uq "/content/drive/My Drive/riskmap_urop/severe" 

In [ ]:
list_ds = tf.data.Dataset.list_files(str("all_*/*"))
list_ds

<ShuffleDataset shapes: (), types: tf.string>

In [ ]:
class_names = ['all_none', 'all_mild', 'all_severe']

def get_label(file_path):
  parts = tf.strings.split(file_path, os.path.sep)
  return parts[-2] == class_names

def decode_img(img):
  img = tf.image.decode_jpeg(img, channels=3)
  img = tf.cast(img, tf.float32)
  # Normalize data
  img = (img/127.5) - 1 
  return tf.image.resize(img, [256, 256])

def process_path(file_path):
  label = get_label(file_path)
  img = tf.io.read_file(file_path)
  img = decode_img(img)
  return img, label

In [ ]:
IMG_SHAPE = (256, 256, 3)

base_model = tf.keras.applications.VGG16(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')

base_model.trainable = False

A local file was found, but it seems to be incomplete or outdated because the auto file hash does not match the original value of 6d6bbae143d832006294945121d1f1fc so we will re-download the data.
58892288/58889256 [==============================] - 0s 0us/step


In [ ]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
prediction_layer = keras.layers.Dense(3, activation="softmax")

In [ ]:
raw_ds = [x.numpy() for x in list(list_ds)]

def image_processer(indices):
    def gen():
          for i in indices:
              file_path = raw_ds[i]
              label = get_label(file_path)
              img = tf.io.read_file(file_path)
              img = decode_img(img)
              yield (img, label)
    return gen

indices = np.arange(len(raw_ds))

In [ ]:
# From https://www.machinecurve.com/index.php/2020/02/18/how-to-use-k-fold-cross-validation-with-keras/

# Define per-fold score containers
acc_per_fold = []
loss_per_fold = []

# # Merge inputs and targets
# inputs = np.concatenate((train_dataset, test_dataset), axis=0)
# # NO TRAIN TARGETS LIST 
# targets = np.concatenate((target_train, target_test), axis=0)

# Define the K-fold Cross Validator
kfold = KFold(n_splits = 4, shuffle=True)

# K-fold Cross Validation model evaluation
fold_no = 1
# NO TARGETS LIST 
for train_indices, test_indices in kfold.split(indices):
  
  current_train_dataset = tf.data.Dataset.from_generator(image_processer(train_indices), (tf.float32, tf.bool), ((256, 256, 3), (3)))
  current_test_dataset = tf.data.Dataset.from_generator(image_processer(test_indices), (tf.float32, tf.bool), ((256, 256, 3), (3)))

  current_train_dataset = current_train_dataset.shuffle(5000).batch(32)
  current_test_dataset = current_test_dataset.shuffle(5000).batch(32)

  # Define the model architecture
  model = tf.keras.Sequential([base_model])

  model.add(tf.keras.layers.Conv2D(32, (3, 3), activation = 'relu'))
  model.add(tf.keras.layers.MaxPooling2D((2, 2), padding = 'same'))
  model.add(tf.keras.layers.Conv2D(64, (3, 3), activation = 'relu'))
  model.add(tf.keras.layers.MaxPooling2D((2, 2), padding = 'same'))

  model.add(tf.keras.layers.Dropout(0.3))
  model.add(global_average_layer)
  model.add(tf.keras.layers.Dense(512, activation = 'sigmoid'))
  model.add(prediction_layer)

  # Compile the model
  base_learning_rate = 0.001
  model.compile(optimizer = tf.keras.optimizers.Adam(lr = base_learning_rate),
                loss='categorical_crossentropy',
                metrics=['accuracy'])


  # Generate a print
  print('------------------------------------------------------------------------')
  print(f'Training for fold {fold_no} ...')

  # Fit data to model
  history = model.fit(current_train_dataset,
                      epochs = 10,
                      steps_per_epoch = 400,
                      validation_data = current_test_dataset, 
                      validation_steps = 100)

  # Generate generalization metrics
  scores = model.evaluate(current_test_dataset, verbose=0) # might need to do x = current_test_dataset
  print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
  acc_per_fold.append(scores[1] * 100)
  loss_per_fold.append(scores[0])

  # Increase fold number
  fold_no = fold_no + 1

# == Provide average scores ==
print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(acc_per_fold)):
  print('------------------------------------------------------------------------')
  print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}%')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
print(f'> Loss: {np.mean(loss_per_fold)}')
print('------------------------------------------------------------------------')

------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/10


UnknownError: ignored

In [ ]:
model.save("cnn_damage.hdf5")
!cp earthquakemodel.hdf5 drive/My\ Drive/riskmap_urop_data

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

In [ ]:
# confusion matrix

# y_predicted = model.predict(list(X_test[0]))
y_predicted = model.predict(current_test_dataset)

In [ ]:
from sklearn.metrics import confusion_matrix

# cm = confusion_matrix(y_test, y_predicted)
cm = confusion_matrix(current_test_dataset, y_predicted)

print(cm)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt 
import seaborn as sn 
plt.figure(figsize = (10, 7))
sn.heatmap(cm, annot = True)
# change label 0 1 2
plt.xlabel('Predicted')
plt.ylabel('Truth')

In [ ]:
from keras.preprocessing.image import img_to_array, load_img
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

def img_show(img):

  img = load_img(img)
  imgplot = plt.imshow(img)
  plt.show()

def img_prediction(img):

  img = load_img(img)
  img = img_to_array(img)
  img = np.expand_dims(img, axis=0)

  img = tf.cast(img, tf.float32)
  img = (img/127.5) - 1
  img = tf.image.resize(img, [256, 256])

  pred = model.predict(img)
 

  damage_dict = {}
  damage_dict['No damage'] = pred[0][0]
  damage_dict['Mild damage'] = pred[0][1]
  damage_dict['Severe damage'] = pred[0][2]

  for key, value in damage_dict.items():
    print(key, ": ", value)
  print("Classification: ", max(damage_dict, key = damage_dict.get))

In [ ]:
!unzip -uq "/content/drive/My Drive/riskmap_urop/puerto_rico_test" 

In [ ]:
path = os.path.expanduser('puerto_rico_test/')

import glob
images = glob.glob(path + '/*.jpg')

for img in images:
  img_show(img)
  img_prediction(img)